In [1]:
from decouple import AutoConfig

dconfig = AutoConfig()

DATABASE_TYPE = dconfig("DATABASE_TYPE")
DATABASE_USER = dconfig("DATABASE_USER")
DATABASE_PASSWORD = dconfig("DATABASE_PASSWORD")
DATABASE_NAME = dconfig("DATABASE_NAME")

In [2]:
DATABASE_HOST = "postgresdb.chdpwgoqvveh.us-east-1.rds.amazonaws.com:5432"

In [3]:
from electrical_demand.database.client import ComplexClient

client = ComplexClient(DATABASE_TYPE, DATABASE_NAME, DATABASE_HOST, DATABASE_USER, DATABASE_PASSWORD)

In [4]:
from electrical_demand.process_data.getters import get_demand



In [5]:
# time-sensitive cross-validation

from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=48,
    max_train_size=10000,
    test_size=1000,
)

# all_splits = list(ts_cv.split(X, y))

In [6]:
from sklearn.model_selection import cross_validate

def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )


In [7]:
import json

with open("data/regions.json", "r") as f:
    f = f.read()
    region_dicts = json.loads(f)

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import SplineTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import RidgeCV
from sklearn.kernel_approximation import Nystroem
import numpy as np
import pandas as pd

categorical_columns = [
    "day_type",
    "weekday"
]

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)


cyclic_spline_transformer = ColumnTransformer(
    transformers=[
        ("categorical", one_hot_encoder, categorical_columns),
        ("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["month"]),
        ("cyclic_weekday", periodic_spline_transformer(7, n_splines=3), ["weekday"]),
        ("cyclic_hour", periodic_spline_transformer(24, n_splines=12), ["hour"]),
    ],
    remainder=MinMaxScaler(),
)

alphas = np.logspace(-6, 6, 25)

cyclic_spline_poly_pipeline = make_pipeline(
    cyclic_spline_transformer,
    Nystroem(kernel="poly", degree=2, n_components=300, random_state=0),
    RidgeCV(alphas=alphas),
)



In [9]:
for region_dict in region_dicts:
    region = region_dict["region"]
    print(region)
    dataset = get_demand(client, region)
    dataset.drop(columns=["region"], inplace=True)
    dataset.drop(columns=["id", "demand_forecast"], inplace=True)
    dataset['month'] = dataset.index.month
    dataset['hour'] = dataset.index.hour
    dataset["weekday"] = dataset.index.weekday
    max_demand = dataset["demand"].max()
    dataset["demand"] = dataset["demand"] / max_demand
    dataset["temperature"].fillna(dataset["temperature_forecast"], inplace=True)
    dataset.sort_index(inplace=True)
    dataset["temperature"] = dataset["temperature"].interpolate(method='linear').ffill()
    dataset.drop(columns=["temperature_forecast"], inplace=True)
    predict_dataset = dataset[dataset["demand"].isnull()]
    train_dataset = dataset[~dataset["demand"].isnull()]

    y = train_dataset["demand"]
    X = train_dataset.drop(columns=["demand"])

    evaluate(cyclic_spline_poly_pipeline, X, y, cv=ts_cv)

    cyclic_spline_poly_pipeline.fit(X, y)

    model = cyclic_spline_poly_pipeline
    predict_dataset["demand_forecast"] = predict_dataset[["day_type", "temperature", "month", "hour", "weekday"]].apply(
            lambda s: model.predict(pd.DataFrame([s]))[0], axis=1
        )
    predict_dataset.drop(columns=["month", "hour", "weekday", "day_type", "temperature", "demand"], inplace=True)
    predict_dataset["demand_forecast"] = predict_dataset["demand_forecast"] * max_demand
    print(predict_dataset.head())

BUENOS AIRES
Mean Absolute Error:     0.034 +/- 0.009
Root Mean Squared Error: 0.043 +/- 0.010


/tmp/ipykernel_25011/4215263569.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset["demand_forecast"] = predict_dataset[["day_type", "temperature", "month", "hour", "weekday"]].apply(
/tmp/ipykernel_25011/4215263569.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset.drop(columns=["month", "hour", "weekday", "day_type", "temperature", "demand"], inplace=True)
/tmp/ipykernel_25011/4215263569.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

                     demand_forecast
datetime                            
2022-11-01 01:00:00      6128.755546
2022-11-01 02:00:00      5610.553361
2022-11-01 03:00:00      5340.731998
2022-11-01 04:00:00      5234.548430
2022-11-01 05:00:00      5198.558816
CATAMARCA
Mean Absolute Error:     0.048 +/- 0.020
Root Mean Squared Error: 0.058 +/- 0.023


/tmp/ipykernel_25011/4215263569.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset["demand_forecast"] = predict_dataset[["day_type", "temperature", "month", "hour", "weekday"]].apply(
/tmp/ipykernel_25011/4215263569.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset.drop(columns=["month", "hour", "weekday", "day_type", "temperature", "demand"], inplace=True)
/tmp/ipykernel_25011/4215263569.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

                     demand_forecast
datetime                            
2022-11-01 01:00:00       121.297558
2022-11-01 02:00:00       109.404970
2022-11-01 03:00:00       100.484370
2022-11-01 04:00:00        95.382080
2022-11-01 05:00:00        93.148690
CHACO
Mean Absolute Error:     0.032 +/- 0.005
Root Mean Squared Error: 0.042 +/- 0.005


...

/tmp/ipykernel_25011/4215263569.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset["demand_forecast"] = predict_dataset[["day_type", "temperature", "month", "hour", "weekday"]].apply(
/tmp/ipykernel_25011/4215263569.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset.drop(columns=["month", "hour", "weekday", "day_type", "temperature", "demand"], inplace=True)
/tmp/ipykernel_25011/4215263569.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

                     demand_forecast
datetime                            
2022-11-01 01:00:00      1139.293031
2022-11-01 02:00:00      1076.755188
2022-11-01 03:00:00      1041.159806
2022-11-01 04:00:00      1029.964189
2022-11-01 05:00:00      1039.039140
SANTIAGO DEL ESTERO
Mean Absolute Error:     0.033 +/- 0.013
Root Mean Squared Error: 0.042 +/- 0.015


/tmp/ipykernel_25011/4215263569.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset["demand_forecast"] = predict_dataset[["day_type", "temperature", "month", "hour", "weekday"]].apply(
/tmp/ipykernel_25011/4215263569.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset.drop(columns=["month", "hour", "weekday", "day_type", "temperature", "demand"], inplace=True)
/tmp/ipykernel_25011/4215263569.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

                     demand_forecast
datetime                            
2022-11-01 01:00:00       128.812100
2022-11-01 02:00:00       111.934148
2022-11-01 03:00:00        98.249957
2022-11-01 04:00:00        97.198531
2022-11-01 05:00:00       102.779824
TUCUMAN
Mean Absolute Error:     0.026 +/- 0.009
Root Mean Squared Error: 0.033 +/- 0.011
                     demand_forecast
datetime                            
2022-11-01 01:00:00       291.988874
2022-11-01 02:00:00       263.164742
2022-11-01 03:00:00       240.533409
2022-11-01 04:00:00       229.351003
2022-11-01 05:00:00       227.040282


/tmp/ipykernel_25011/4215263569.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset["demand_forecast"] = predict_dataset[["day_type", "temperature", "month", "hour", "weekday"]].apply(
/tmp/ipykernel_25011/4215263569.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataset.drop(columns=["month", "hour", "weekday", "day_type", "temperature", "demand"], inplace=True)
/tmp/ipykernel_25011/4215263569.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =